# QAOA CON QISKIT

En este notebook, aprenderemos a utilizar las funcionalidades de Qsikit para manejar problemas de optimización y resolverlos con ayuda del algoritmo QAOA

Comenzamos importando los paquetes que vamos a utilizar

In [ ]:
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.converters import InequalityToEquality
from qiskit_optimization.converters import IntegerToBinary
from qiskit_optimization.converters import LinearEqualityToPenalty
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms.optimizers import COBYLA

from qiskit import Aer
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.quantum_info import Pauli
from qiskit.opflow.primitive_ops import PauliOp
from qiskit.opflow.list_ops import SummedOp
from qiskit.opflow import I, X, Y, Z

## Creando el hamiltoniano directamente

Podemos resolver problemas en Qiskit con QAOA a partir de un hamiltoniano del que ya dispongamos o que creemos explícitamente. Como ejemplo, vamos a crear un hamiltoniano sencillo y encontrar su estado de mínima energía con QAOA

In [ ]:
op = PauliOp(Pauli("ZZ")) # Equivale a Pauli(([1,1],[0,0])). 
                          # El primer array indica posiciones de las Zs y el segundo, de las Xs
print("El hamiltoniano es",op)

qaoa = QAOA(optimizer = COBYLA(), quantum_instance=Aer.get_backend('aer_simulator'), reps = 1)  
result = qaoa.compute_minimum_eigenvalue(op)
print(result)

Ahora, probamos con un problema más complicado

In [ ]:
op = -PauliOp(Pauli("ZZ")) -0.5*PauliOp(Pauli("ZI")) - PauliOp(Pauli("IZ"))
# Otra forma de hacer lo mismo op = SummedOp([PauliOp(Pauli("ZZ"),-1),PauliOp(Pauli("ZI"),-0.5),PauliOp(Pauli("IZ"),-1)])
print("El hamiltoniano es",op)
result = qaoa.compute_minimum_eigenvalue(op)
print(result)

Probamos aumentando el valor de $p$

In [ ]:
qaoa = QAOA(optimizer = COBYLA(), quantum_instance=Aer.get_backend('aer_simulator'), reps = 2)  
result = qaoa.compute_minimum_eigenvalue(op)
print(result)

Probemos con un problema con más qubits

In [ ]:
# Corresponde al problema de D-Wave con J = {(0,1):1,(0,2):1,(1,2):1,(1,3):1,(2,4):1,(3,4):1}

op = (Z^Z^I^I^I) + (Z^I^Z^I^I) + (I^Z^Z^I^I) + (I^Z^I^Z^I) + (I^I^Z^I^Z) + (I^I^I^Z^Z) # ¡Paréntesis necesarios!
print("El hamiltoniano es", op)
qaoa = QAOA(optimizer = COBYLA(), quantum_instance=Aer.get_backend('aer_simulator'), reps = 1)  
result = qaoa.compute_minimum_eigenvalue(op)
print(result)

## Trabajando con problemas cuadráticos

Qiskit proporciona herramientas para definir problemas cuadráticos y convertirlos en hamiltonianos que podemos usar con QAOA. También define clases que podemos usar para resolver estos problemas de forma transparente. Veamos cómo funcionan. 

Definimos un problema cuadrático con restricciones

In [ ]:
qp = QuadraticProgram()
qp.binary_var('x')
qp.binary_var('y')
qp.binary_var('z')

# Añadimos el objetivo

qp.minimize(linear = {'y':-1}, quadratic = {('x','y'):2, ('z','y'):-4})

# Añadimos una restricción lineal

qp.linear_constraint(linear = {'x':1, 'y':2, 'z':3}, sense ="<=", rhs = 5)

print(qp)

Ahora, podemos resolver este problema de forma exacta

In [ ]:
np_solver = NumPyMinimumEigensolver()
np_optimizer = MinimumEigenOptimizer(np_solver)
result = np_optimizer.solve(qp)
print(result)

Podemos resolver el problema también con QAOA

In [ ]:
qaoa_optimizer = MinimumEigenOptimizer(qaoa)
result = qaoa_optimizer.solve(qp)
print(result)

Para obtener más información sobre los resultados:

In [ ]:
print('Orden de las variables:', [var.name for var in result.variables])
for s in result.samples:
    print(s)

Y para obtener la misma información que con el QAOA:

In [ ]:
print(result.min_eigen_solver_result)

# El proceso en más detalle

La conversión que se hace internamente sigue varios pasos. El primero es convertir las restricciones de desiguladades en restricciones de igualdad

In [ ]:
ineq_to_eq = InequalityToEquality()
qp_eq = ineq_to_eq.convert(qp)
print(qp_eq)

A continuación, se convierten las variables enteras en binarias.

In [ ]:
int_to_bin = IntegerToBinary()
qp_bin = int_to_bin.convert(qp_eq)
print(qp_bin)

Ahora, se convierten las restricciones en penalizaciones en el objetivo

In [ ]:
eq_to_pen = LinearEqualityToPenalty()
qubo = eq_to_pen.convert(qp_bin)
print(qubo)

Podríamos hacer lo mismo en un solo paso

In [ ]:
qp_to_qubo = QuadraticProgramToQubo()
qubo2 = qp_to_qubo.convert(qp)
print(qubo2)

Finalmente, podemos recuperar el hamiltoniano correspondiente al problema

In [ ]:
op, offset = qubo.to_ising()
print("El hamiltoniano es", op)
print("Y la constante es", offset)

Y usar QAOA directamente sobre este operador

In [ ]:
result = qaoa.compute_minimum_eigenvalue(op)
print(result)